In [2]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 2s (228 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [3]:
from IPython.display import clear_output
!ollama pull llama2
#clear_output()

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling

In [5]:
!pip install langchain-ollama

from langchain_ollama.llms import OllamaLLM
#from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama2"
model = OllamaLLM(model=MODEL)
#embeddings = OllamaEmbeddings()

response = model.invoke("Tell me a joke")
print(response)


Sure, here's one:

Why don't scientists trust atoms?
Because they make up everything!

I hope you found that amusing! Do you want to hear another one?


In [6]:
response = model.invoke("Who won presidential election 2024?")
print(response)


I'm just an AI, I don't have access to real-time information or events that have not yet occurred. The presidential election in 2024 has not yet taken place, and the outcome is uncertain. The next presidential election is scheduled to take place in November 2024, and it will be decided by the American people through a democratic process.

It's important to note that predicting the outcome of an election is inherently difficult and can be influenced by a complex array of factors, including the candidates, their campaigns, and external events. Additionally, polling and forecasting models are subject to uncertainties and limitations, and their predictions may not always accurately reflect the final outcome of an election.

In summary, I cannot provide you with information on who will win the 2024 presidential election as it has not yet taken place and the outcome is uncertain. The best way to stay informed about the election is to follow reliable news sources and to consult with reputabl

In [14]:
#!pip install langchain-community
#!pip install pypdf
from langchain_community.document_loaders import PyPDFLoader


pdf_files = ["machine-learning.pdf"]

all_pages = []
for pdf_file in pdf_files:
    loader = PyPDFLoader("/machine-learning.pdf")
    pages = loader.load_and_split()
    all_pages.extend(pages)

In [15]:
all_pages

[Document(metadata={'source': '/machine-learning.pdf', 'page': 0}, page_content="Explore IBM's machine learning solution\nSign up for AI updates\nHome / Topics / Machine learning\nWhat is machine learning (ML)?\n10/26/24, 12:21 AM What Is Machine Learning (ML)? | IBM\nhttps://www.ibm.com/topics/machine-learning 1/14"),
 Document(metadata={'source': '/machine-learning.pdf', 'page': 1}, page_content="What is machine learning?\nMachine learning (ML) is a branch of\xa0\n\xa0and computer science that focuses on the using data and algorithms to\nenable AI to imitate the way that humans learn, gradually improving its\naccuracy.\nHow does machine learning work?\nIBM TechXchange Conference 2024 | \xa0\nSee what IBM's community of technology advocates have been working on, read the latest product\nannouncements, and watch replays of the event\nWhat is machine learning?\nMachine learning versus deep learning versus neural networks\nMachine learning\xa0methods\nReinforcement machine learning\nComm

In [16]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know."

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't answer the question, reply "I don't know."

Context: Here is some context

Question: Here is a question



In [ ]:
chain = prompt | model | parser

In [ ]:
chain.input_schema.schema()

In [ ]:
chain.invoke(
    {
        "context": "The name I was given was Santiago",
        "question": "What's my name?"
    }
)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_milvus import Milvus

vectorstore = Milvus.from_documents(
    documents=all_pages,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": 19530}
)

In [ ]:
retriever = vectorstore.as_retriever()

retriever.invoke("Machine Learning")

In [ ]:
from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "What is Machine Learning according to the document?",
    "How does machine learning work according to the document?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()